### 確認の処理を加えたコードです

# import

In [ ]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd

tf.__version__

In [ ]:
%matplotlib inline

In [ ]:
def disp_datainfo2(name, data):
    print(name, ":", "shape:", data.shape, "max:", np.max(data), "min:", np.min(data))
    
def disp_datainfo():
    disp_datainfo2("x_train", x_train)
    disp_datainfo2("y_train", y_train)
    disp_datainfo2("x_test", x_test)
    disp_datainfo2("y_test", y_test)

# データの読み込み

In [ ]:
# MNIST データのロード
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
#(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

In [ ]:
disp_datainfo() # shape, max, min

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(x_train[i], cmap=plt.cm.binary)
    plt.xlabel(str(y_train[i]))

In [ ]:
n = 10
indexes = [np.where(y_train==x)[0][:n] for x in range(10)]

for x in range(10):
    plt.figure(figsize=(10,2))
    for i, idx in enumerate(indexes[x]):
        img_arr = np.asarray(x_train[idx])
        plt.subplot(1,n,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(img_arr, cmap=plt.cm.binary)
plt.show()

In [ ]:
# 0 - 255の画素データを 0.0 - 1.0に変換
x_train = x_train / 255.0
x_test = x_test / 255.0

# 2次元画素データを1次元に変換
x_train = x_train.reshape(-1, 784) 
x_test = x_test.reshape(-1, 784)

# one-hotベクトル化
y_train = keras.utils.to_categorical(y_train, 10) 
y_test = keras.utils.to_categorical(y_test, 10)

In [ ]:
disp_datainfo() # shape, max, min

# モデルの構築

In [ ]:
from tensorflow.keras.layers import Dense

class MLP(keras.Model):

    def __init__(self):
        super(MLP, self).__init__()

        self.dense1 = Dense(64, activation='relu')
        self.dense2 = Dense(10, activation='softmax')

    def call(self, inputs):
        x = self.dense1(inputs)
        return self.dense2(x)

model = MLP()

# コンパイル

In [ ]:
model.compile(optimizer=keras.optimizers.SGD(lr=0.5),
              loss='mean_squared_error',
              metrics=['accuracy'])

# 訓練

In [ ]:
EPOCHS = 15

In [ ]:
history = model.fit(x_train, y_train, epochs=EPOCHS, validation_split=0.2)

In [ ]:
acc = 'acc'
if tf.__version__[0]=='2':
    acc='accuracy'

plt.plot(range(1, EPOCHS+1), history.history[acc], label="training")
plt.plot(range(1, EPOCHS+1), history.history['val_'+acc], label="validation")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(range(1, EPOCHS+1), history.history['loss'], label="training")
plt.plot(range(1, EPOCHS+1), history.history['val_loss'], label="validation")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# 評価

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)

In [ ]:
print(test_loss)
print(test_acc)

# 推論

In [ ]:
# 実際は訓練、評価に使っていないデータを使ってください
predicts = model.predict(x_test)

In [ ]:
for x in range(10):
    plt.figure(figsize=(6,2))
    plt.subplot(1,2,1)
    print(np.argmax(predicts[x]))
    plt.imshow(x_test[x].reshape(28,28), cmap=plt.cm.binary)
    plt.xticks([])
    plt.yticks([])
    plt.subplot(1,2,2)
    plt.bar([str(x) for x in range(10)], predicts[x])
    plt.show()


# 混合行列（confusion matrix）

In [ ]:
predicts2 = np.argmax(predicts,1)
y_test2 = np.argmax(y_test,1)

In [ ]:
cmat = {}
for y in range(10):  # 正解 
  for x in range(10):  # 予測
    cmat[x, y] = np.count_nonzero(predicts2[indexes[x]]==y)
  cmat[y, "sum"] = len(predicts2[indexes[y]])

for x in range(10):
    sum=0
    for y in range(10):
        sum += cmat[y, x]
    cmat["sum", x] = sum

cmat["sum", "sum"] = len(y_test2)



In [ ]:
pd.Series(cmat).unstack()

# 縦が正解ラベル、横が推論値

# 間違った例

In [ ]:
for x in np.where(y_test2!=predicts2)[0][:10]:
    plt.figure(figsize=(6,2))
    plt.subplot(1,2,1)
    print(y_test2[x], '->', predicts2[x])
    plt.imshow(x_test[x].reshape(28,28), cmap=plt.cm.binary)
    plt.xticks([])
    plt.yticks([])
    plt.subplot(1,2,2)
    plt.bar([str(x) for x in range(10)], predicts[x])
    plt.show()